In [ ]:
!pip install --upgrade google-search-results


  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=b11cf7c3c1bb91d754e6a56b6ac8b724ceb9d061b1676defea1275f901d9bd4f
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
from serpapi import GoogleSearch


Test code


In [ ]:
from serpapi import GoogleSearch

print("✅ SerpAPI installed & imported correctly!")

SERPAPI_KEY = "5d6ee01b24626f8a1960777e8c160a975be6b62d1016b199156e529dc2e9bdf7"

params = {
    "engine": "google",
    "q": "daily horoscope for libra",
    "api_key": SERPAPI_KEY,
    "num": 3
}

search = GoogleSearch(params)
results = search.get_dict()

if "organic_results" in results:
    for r in results["organic_results"]:
        print("🔗", r.get("title"), "-", r.get("link"))
else:
    print("⚠️ No results or invalid API key")



✅ SerpAPI installed & imported correctly!
🔗 Libra Horoscopes: Daily & Today - https://www.horoscope.com/us/horoscopes/general/horoscope-general-daily-today.aspx?sign=7
🔗 Libra Daily Horoscope Yahoo Life Astrology: July 14, 2025 - https://www.yahoo.com/lifestyle/horoscope/libra/daily-20250714/
🔗 Libra Daily Horoscope Today: July 14, 2025 - https://www.elle.com/horoscopes/daily/a103/libra-daily-horoscope/


In [ ]:

!pip install streamlit flatlib transformers pyngrok

app_code = '''
import streamlit as st
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib import const
from serpapi import GoogleSearch
from transformers import pipeline

# Streamlit setup
st.set_page_config(page_title="SoulScope AI", page_icon="🔮")
st.title("🔮 SoulScope – Real-Time Horoscope Summarizer")

import datetime
from transformers import pipeline

def clean_sentence_end(text):
    """Ensure the summary ends with a complete sentence."""
    # Find the last sentence-ending punctuation
    for p in [".", "!", "?"]:
        if p in text:
            text = text[:text.rfind(p)+1]  # cut after the last punctuation
            break
    return text.strip()

def summarize_snippets(snippets):
    if not snippets:
        return "⚠️ No relevant horoscope info found online."

    # ✅ Filter out irrelevant SEO/Navigation junk
    filtered_snippets = [
        s for s in snippets
        if not any(x in s.lower() for x in ["hub", "page", "click", "navigation", "free astrology", "explore"])
    ]

    if not filtered_snippets:
        return "⚠️ No useful horoscope data found."

    # ✅ Combine & truncate safely
    text = " ".join(filtered_snippets)
    if len(text) > 1800:
        text = text[:1800]

    # ✅ Add mystical astrologer persona prompt
    mystical_prompt = f"""
You are a mystical astrologer. From the following raw horoscope info, craft a beautiful,
poetic, and inspiring horoscope message. Do NOT mention websites or technical text.

{text}
"""

    # ✅ Summarize safely
    try:
        summary = summarizer(
            mystical_prompt,
            max_length=220,   # allow slightly more words
            min_length=80,    # ensures it's not too short
            do_sample=False
        )[0]['summary_text']
    except Exception as e:
        return f"⚠️ Could not summarize: {e}"

    # ✅ Post-process junk removal
    bad_keywords = ["hub page", "click", "website", "monthly hub", "astrology page"]
    if any(k in summary.lower() for k in bad_keywords):
        summary = "✨ The cosmic energies invite you to embrace change, find balance, and trust the unfolding journey. 🌙"
    else:
        # ✅ Ensure the summary ends with a complete sentence
        summary = clean_sentence_end(summary)

    return summary


# User inputs
name = st.text_input("Your Name")
birth_date = st.date_input(
    "Birth Date",
    min_value=datetime.date(1800, 1, 1),
    value=datetime.date(2000, 1, 1))
birth_time = st.time_input("Birth Time (24hr)")
location_input = st.text_input("Birth City", "Hyderabad")
mode = st.radio("Choose Horoscope Type", ["Daily", "Weekly", "Monthly"])

st.markdown("Fill in your birth details to get a unique horoscope created from real-time web insights 🌐")

# Hugging Face summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

SERPAPI_KEY = "5d6ee01b24626f8a1960777e8c160a975be6b62d1016b199156e529dc2e9bdf7"

# Fetch snippets via SerpAPI
def fetch_astrology_snippets(query):
    params = {
        "engine": "google",
        "q": query,
        "api_key": SERPAPI_KEY,
        "num": 5
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    snippets = []
    if "organic_results" in results:
        for r in results["organic_results"]:
            if "snippet" in r:
                snippets.append(r["snippet"])
    return snippets

# Generate horoscope
def generate_real_time_horoscope(sun_sign, mode):
    query = f"{mode} horoscope for {sun_sign} zodiac sign astrology"
    snippets = fetch_astrology_snippets(query)
    return summarize_snippets(snippets)

if st.button("🔭 Reveal Horoscope"):
    if not name:
        st.warning("⚠️ Please enter your name.")
    else:
        try:
            date_str = birth_date.strftime('%Y/%m/%d')
            time_str = birth_time.strftime('%H:%M')

            # Default to Hyderabad
            if "hyderabad" in location_input.lower():
                geo = GeoPos('+17:23', '+78:29')
            else:
                geo = GeoPos('0', '0')

            chart = Chart(Datetime(date_str, time_str, '+05:30'), geo)
            sun_sign = chart.get(const.SUN).sign
            st.markdown(f"### ✨ Real-Time {mode} Horoscope for {name} ({sun_sign})")

            horoscope = generate_real_time_horoscope(sun_sign, mode)
            st.write(horoscope)

        except Exception as e:
            st.error(f"Error: {e}")
'''

with open("soulscope_app.py", "w") as f:
    f.write(app_code)

# Launch ngrok + streamlit
from pyngrok import ngrok
import os
!ngrok config add-authtoken 2zbavHJjAIgI72TZRxjx4iUr22m_yectMkrmuqDLntNyUiAu

ngrok.kill()
public_url = ngrok.connect(8501)
os.system("streamlit run soulscope_app.py &")
print("🌐 Your SoulScope is live at:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Your SoulScope is live at: NgrokTunnel: "https://deba58e13da9.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pkill -f ngrok
!pkill -f streamlit
